## Import Libraries

In [9]:
import os
import re
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from tqdm import tqdm

In [10]:
def sorted_alphanumeric(data):
    convert = lambda text: int(text) if text.isdigit() else text.lower()
    alphanum_key = lambda key: [convert(c) for c in re.split('([0-9]+)', key)]
    return sorted(data, key=alphanum_key)

In [11]:
# Define the dataset class
class ColorizationDataset(Dataset):
    def __init__(self, color_path, gray_path, size=160):
        self.size = size
        self.color_path = color_path
        self.gray_path = gray_path
        self.color_images = self.load_images(color_path)
        self.gray_images = self.load_images(gray_path)

    def load_images(self, path):
        images = []
        files = sorted_alphanumeric(os.listdir(path))
        for i in tqdm(files):
            if i == '6000.jpg':
                break
            else:
                img = Image.open(os.path.join(path, i)).convert('RGB')
                img = img.resize((self.size, self.size), Image.ANTIALIAS)
                img = np.array(img) / 255.0
                images.append(img)
        return images

    def __len__(self):
        return len(self.color_images)

    def __getitem__(self, idx):
        color_img = transforms.ToTensor()(self.color_images[idx])
        gray_img = transforms.ToTensor()(self.gray_images[idx])
        return gray_img, color_img

In [12]:
class ColorizationModel(nn.Module):
    def __init__(self):
        super(ColorizationModel, self).__init__()
        # Define your layers here
        self.downsample1 = self.downsample(3, 128, kernel_size=3, apply_batch_normalization=False)
        self.downsample2 = self.downsample(128, 128, kernel_size=3, apply_batch_normalization=False)
        self.downsample3 = self.downsample(128, 256, kernel_size=3, apply_batch_normalization=True)
        self.upsample1 = self.upsample(256, 128, kernel_size=3, dropout=False)
        self.upsample2 = self.upsample(128, 128, kernel_size=3, dropout=False)
        self.upsample3 = self.upsample(128, 3, kernel_size=3, dropout=False)
        self.upsample4 = self.upsample(3, 3, kernel_size=2, dropout=False)
        self.conv_output = nn.Conv2d(3, 3, kernel_size=3, padding=1).to(torch.double)
        self.upsample_output = nn.Upsample((160, 160), mode='bilinear', align_corners=False)

    def downsample(self, in_channels, out_channels, kernel_size, apply_batch_normalization=True):
        padding = (kernel_size - 1) // 2
        layers = []
        layers.append(nn.Conv2d(in_channels, out_channels, kernel_size, padding=padding, stride=2).to(torch.float32))
        if apply_batch_normalization:
            layers.append(nn.BatchNorm2d(out_channels).to(torch.float32))
        layers.append(nn.LeakyReLU(0.2).to(torch.float32))
        return nn.Sequential(*layers)

    def upsample(self, in_channels, out_channels, kernel_size, dropout=False):
        layers = []
        layers.append(nn.ConvTranspose2d(in_channels, out_channels, kernel_size, padding=1, stride=2, output_padding=1))
        if dropout:
            layers.append(nn.Dropout(0.2))
        layers.append(nn.LeakyReLU(0.2))
        return nn.Sequential(*layers)

    def forward(self, x):
        d1 = self.downsample1(x)
        d2 = self.downsample2(d1)
        d3 = self.downsample3(d2)

        u1 = self.upsample1(d3) + d2
        u2 = self.upsample2(u1) + d1
        u3 = self.upsample3(u2) + x
        u4 = self.upsample4(u3)

        # Adjust the final convolutional layer to match the target size
        conv_output = self.conv_output(u4)
        upsample_output = self.upsample_output(conv_output)

        return upsample_output

In [ ]:
# Instantiate the dataset and model
color_dataset = ColorizationDataset(color_path=r'C:\Users\ulas_\OneDrive\Masaüstü\Deep Learning Project\landscape Images\color',
                                    gray_path=r'C:\Users\ulas_\OneDrive\Masaüstü\Deep Learning Project\landscape Images\gray')

train_size = int(0.8 * len(color_dataset))
test_size = len(color_dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(color_dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=50, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=50, shuffle=False)

model = ColorizationModel().to(torch.double)

  0%|                                                                                         | 0/7129 [00:00<?, ?it/s]<ipython-input-11-4324545365c3>:18: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  img = img.resize((self.size, self.size), Image.ANTIALIAS)
 32%|█████████████████████████▎                                                    | 2311/7129 [00:24<00:50, 94.65it/s]

In [6]:
# Define the loss function and optimizer
criterion = nn.L1Loss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [7]:
print("Inputs shape:", inputs.shape)
print("Outputs shape:", outputs.shape)
print("Targets shape:", targets.shape)

NameError: name 'inputs' is not defined

In [8]:
# Training the model
num_epochs = 50
for epoch in range(num_epochs):
    total_loss = 0
    total_batches = 0

    for batch_idx, (inputs, targets) in enumerate(train_loader):
        optimizer.zero_grad()
        
        # Convert inputs to double precision
        inputs = inputs.to(torch.double)
        
        # Print shapes
        print("Batch:", batch_idx + 1)
        print("Inputs shape:", inputs.shape)
        print("Targets shape:", targets.shape)

        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        total_batches += 1

    average_loss = total_loss / total_batches
    print(f"Epoch {epoch + 1}/{num_epochs}, Batches: {total_batches}, Average Loss: {average_loss}")

# Training completed


KeyboardInterrupt: 

In [ ]:
# Evaluate the model on the test set
model.eval()
test_losses = []
with torch.no_grad():
    for inputs, targets in test_loader:
        outputs = model(inputs)
        test_loss = criterion(outputs, targets)
        test_losses.append(test_loss.item())

print("Mean Test Loss:", np.mean(test_losses))

In [ ]:
# Plotting colorized images
def plot_images(color, grayscale, predicted):
    plt.figure(figsize=(15, 15))
    plt.subplot(1, 3, 1)
    plt.title('Color Image', color='green', fontsize=20)
    plt.imshow(color.permute(1, 2, 0))
    plt.subplot(1, 3, 2)
    plt.title('Grayscale Image ', color='black', fontsize=20)
    plt.imshow(grayscale.permute(1, 2, 0))
    plt.subplot(1, 3, 3)
    plt.title('Predicted Image ', color='Red', fontsize=20)
    plt.imshow(predicted.permute(1, 2, 0))

for i in range(50, 58):
    inputs, targets = test_dataset[i]
    inputs = inputs.unsqueeze(0)  # Add batch dimension
    targets = targets.unsqueeze(0)
    predicted = model(inputs)
    plot_images(targets, inputs, predicted)

plt.show()